In [ ]:
from build_dataset import MRILesionDatasetBuilder

# Build the dataset for the lesion inpaiting model
- Train / test split
- Filter out slices with no lesions

Others
- Include SHIFTS dataset

In [ ]:
dataset_builder = MRILesionDatasetBuilder()
dataset_builder.build_dataset()

Skipping empty mask for train 640 at slice 95
Skipping empty mask for train 640 at slice 96
Skipping empty mask for train 640 at slice 97
Skipping empty mask for train 644 at slice 86
Skipping empty mask for train 644 at slice 87
Skipping empty mask for train 644 at slice 88
Skipping empty mask for train 644 at slice 89
Skipping empty mask for train 644 at slice 90
Skipping empty mask for train 644 at slice 91
Skipping empty mask for train 644 at slice 92
Skipping empty mask for train 644 at slice 93
Skipping empty mask for train 644 at slice 94
Skipping empty mask for train 644 at slice 95
Skipping empty mask for train 644 at slice 96
Skipping empty mask for train 644 at slice 97
Skipping empty mask for train 658 at slice 85
Skipping empty mask for train 658 at slice 86
Skipping empty mask for train 658 at slice 87
Skipping empty mask for train 658 at slice 88
Skipping empty mask for train 658 at slice 89
Skipping empty mask for train 658 at slice 90
Skipping empty mask for train 658 